Pakiety

In [ ]:
import pandas as pd 
import librosa

import numpy as np
import os
import torch
import time
from google.colab import drive
from numpy import save
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score


Połączenie do Drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


Wczytujemy dane do modelowania

In [ ]:
statusy = pd.read_csv('/content/drive/My Drive/Projekt_ptaki/statusy.csv', encoding='cp1250')
y = np.array(statusy['species'])
X = np.load("drive/My Drive/Projekt_ptaki/cechy.npy")

Usuwamy puste obserwacje (wypełnione wcześniej zerami)

In [ ]:
do_wykl = np.where(~X.any(axis=1))[0]
X = np.delete(X, do_wykl, axis=0)
y = np.delete(y, do_wykl, axis=0)

Wybieramy 50 najlepszych cech (gdy bierzemy więcej, moc nie wzrasta, a tworzy się overfitting)

In [ ]:
selector = SelectKBest(f_classif, k=100)
selected_features = selector.fit_transform(X, y)

Podział train - test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.3, random_state=1234) #, stratify = y)

SMOTE - do zbilansowania próby

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

Konwertujemy zmienną zależną do postaci kategorycznej

In [ ]:
lb = LabelEncoder()
y_train_encod = to_categorical(lb.fit_transform(y_train))
y_test_encod = to_categorical(lb.fit_transform(y_test))

Skalujemy zmienne niezależne

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

Standardowy Multilayer Perceptron

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(512, input_dim = X_train.shape[1], activation = 'relu')) #128
model.add(Dropout(0.6)) #0.4
model.add(Dense(256, activation = 'relu')) #64
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=50)

In [ ]:
history = model.fit(X_train, y_train_encod, batch_size=256, epochs=500, 
                    validation_data=(X_test, y_test_encod),
                    callbacks=[early_stop])

Tworzymy predykcje z modelu

In [ ]:
predictions_train = model.predict_classes(X_train)
predictions_test = model.predict_classes(X_test)

predictions_train = lb.inverse_transform(predictions_train)
predictions_test = lb.inverse_transform(predictions_test)


Accuracy na zbiorze train

In [ ]:
(1-round(sum(y_train != predictions_train)/len(y_train),3))*100

88.8

Accuracy na zbiorze testowym

In [ ]:
(1-round(sum(y_test != predictions_test)/len(y_test),3))*100

48.1

F1 Score na zbiorze testowym

In [ ]:
f1_score(y_test, predictions_test, average = 'micro') 

0.4806701030927835

Zapis modelu

In [ ]:
model.save('drive/My Drive/Projekt_ptaki/model_cnn.h5')

Wczytanie modelu

In [ ]:
model = keras.models.load_model('drive/My Drive/Projekt_ptaki/model_cnn.h5')

LSTM

In [ ]:
from tensorflow.keras.layers import LSTM
model_new = Sequential()
model_new.add(LSTM(units=256, dropout=0.6, recurrent_dropout=0.20, return_sequences=True,input_shape = (1, X_train.shape[1])))
model_new.add(LSTM(units=128, dropout=0.5, recurrent_dropout=0.20, return_sequences=False))
model_new.add(Dense(30, activation='softmax'))
model_new.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['acc','mse', 'mae', 'mape'])
early_stop = EarlyStopping(monitor='val_loss', patience=50)

Konwersja danych do wymiarów potrzebnych do zbudowania LSTM

In [ ]:
X_train_remod = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_remod = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

Trenowanie modelu

In [ ]:
history_new = model_new.fit(X_train_remod, y_train_encod, batch_size=256, epochs=250, 
                    validation_data=(X_test_remod, y_test_encod),
                    callbacks=[early_stop])

Tworzymy predykcje z modelu

In [ ]:
predictions_train = model_new.predict_classes(X_train_remod)
predictions_test = model_new.predict_classes(X_test_remod)

predictions_train = lb.inverse_transform(predictions_train)
predictions_test = lb.inverse_transform(predictions_test)

Accuracy na zbiorze train

In [ ]:
(1-round(sum(y_train != predictions_train)/len(y_train),3))*100

53.49999999999999

Accuracy na zbiorze testowym

In [ ]:
(1-round(sum(y_test != predictions_test)/len(y_test),3))*100

34.699999999999996

F1 Score na zbiorze testowym

In [ ]:
f1_score(y_test, predictions_test, average = 'micro') 

0.34664948453608246

Zapis modelu

In [ ]:
model_new.save('drive/My Drive/Projekt_ptaki/model_lstm_1.h5')

In [ ]:
#https://medium.com/analytics-vidhya/adding-noise-to-audio-clips-5d8cee24ccb8
#https://becominghuman.ai/musical-instrument-sound-classification-using-cnn-part-2-2-aaa668a3862a